In [1]:
from detr_config import Config
from detr_dataset import DataGenerator
from detr_trainer import get_trainer
from detr_evaluation import ModelEvaluator
from detr_model import ModelGenerator
from detr_file_manager import FileManager
from detr_metrics import MetricsAggregator

In [2]:

Config.set_local_settings()

for architecture, num_queries, transformer_layers in Config.HYPERPARAMS:
    
    model_generator = ModelGenerator(architecture, num_queries, transformer_layers)
    image_processor = model_generator.get_image_processor()
    file_manager = FileManager(model_generator)
    data_generator = DataGenerator(image_processor, file_manager)
    
    file_manager.clean_model_logs()
        
    # -----------------------
    # K-fold Cross Validation 
    # -----------------------
    
    metrics_aggregator = MetricsAggregator(file_manager)

    for fold in range(1,11):

        # Model
        
        model = model_generator.get_model()
        
        # Dataset
        
        file_manager.set_validation_fold_setup(fold)
        
        train_dataset, train_loader = data_generator.get_train_dataset_dataloader()
        valid_dataset, valid_loader = data_generator.get_valid_dataset_dataloader()

        # Training
        
        fold_version = file_manager.get_fold_version()
        trainer = get_trainer(fold_version)
        trainer.fit(model, train_loader, valid_loader)
        
        # Validation
        
        checkpoints_dir = file_manager.get_checkpoints_dir()
        model_evaluator = ModelEvaluator(checkpoints_dir, image_processor, valid_dataset, valid_loader)
        valid_metrics = model_evaluator.get_metrics(threshold=Config.THRESHOLD)
        
        metrics_aggregator.add_valid_metrics(valid_metrics)
        metrics_aggregator.save_metrics()
        
        file_manager.clean_checkpoints()

        break # Fold
    
    # Aggregate K-Fold Metrics
    
    metrics_aggregator.calculate_valid_mean()
    metrics_aggregator.save_metrics()
    
    # -----------------------
    # Testing
    # -----------------------
    
    # Model
    
    model = model_generator.get_model() 

    # Dataset
    
    file_manager.set_training_setup()

    train_valid_dataset, train_valid_loader = data_generator.get_train_valid_dataset_dataloader()
    test_dataset, test_loader = data_generator.get_test_dataset_dataloader()
    
    # Training
    
    test_version = file_manager.get_test_version()
    trainer = get_trainer(test_version)
    trainer.fit(model, train_valid_loader, test_loader)
    
    # Testing

    checkpoints_dir = file_manager.get_checkpoints_dir()
    model_evaluator = ModelEvaluator(checkpoints_dir, image_processor, test_dataset, test_loader)
    test_metrics = model_evaluator.get_metrics(threshold=Config.THRESHOLD)
    
    # Metrics
    
    metrics_aggregator.add_test_metrics(test_metrics)
    metrics_aggregator.save_metrics()
    
    # break # Hyperparameter

The `max_size` parameter is deprecated and will be removed in v4.26. Please specify in `size['longest_edge'] instead`.


Model Name:  model=DETR_queries=100_layers=6


Some weights of the model checkpoint at facebook/detr-resnet-50 were not used when initializing DetrForObjectDetection: ['model.backbone.conv_encoder.model.layer4.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer2.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer3.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer1.0.downsample.1.num_batches_tracked']
- This IS expected if you are initializing DetrForObjectDetection from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DetrForObjectDetection from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DetrForObjectDetection were not initialized from the model checkpoin

Model Name:  model=DETR_queries=100_layers=6
Loading Annotations from:  InBreast_Coco\fold_1\train\_annotations.coco.json
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
Loading Annotations from:  InBreast_Coco\fold_1\valid\_annotations.coco.json
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!


GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name | Type                   | Params
------------------------------------------------
0 | detr | DetrForObjectDetection | 41.5 M
------------------------------------------------
41.3 M    Trainable params
222 K     Non-trainable params
41.5 M    Total params
166.007   Total estimated model params size (MB)


...

Accumulating evaluation results...
DONE (t=0.01s).


Some weights of the model checkpoint at facebook/detr-resnet-50 were not used when initializing DetrForObjectDetection: ['model.backbone.conv_encoder.model.layer4.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer2.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer3.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer1.0.downsample.1.num_batches_tracked']
- This IS expected if you are initializing DetrForObjectDetection from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DetrForObjectDetection from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DetrForObjectDetection were not initialized from the model checkpoin

Model Name:  model=DETR_queries=100_layers=6
Loading Annotations from:  InBreast_Coco\train_valid\_annotations.coco.json
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
Loading Annotations from:  InBreast_Coco\test\_annotations.coco.json
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!



  | Name | Type                   | Params
------------------------------------------------
0 | detr | DetrForObjectDetection | 41.5 M
------------------------------------------------
41.3 M    Trainable params
222 K     Non-trainable params
41.5 M    Total params
166.007   Total estimated model params size (MB)


<generator object Module.named_parameters at 0x00000209296F8E40>
